### 載入MNIST手寫數字數據集

In [1]:
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("MNIST_data/",one_hot = True)

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
import tensorflow as tf

### 定義神經網路函式

In [3]:
def weight_variable(shape,name):
    return tf.Variable(tf.truncated_normal(shape,stddev = 0.1),name = name)

def bias_variable(shape,name):
    return tf.Variable(tf.constant(0.1,shape = shape),name = name)

### 定義變數

In [4]:
X = tf.placeholder(tf.float32,[None,784])

### 定義模型

#### 第一層(encoding)

In [5]:
w1 = weight_variable([784,300],'w1')
b1 = bias_variable([300],'b1')
o1 = tf.nn.relu(tf.matmul(X,w1)+b1)

#### 第二層(encoding)

In [6]:
w2 = weight_variable([300,100],'w2')
b2 = bias_variable([100],'b2')
o2 = tf.nn.relu(tf.matmul(o1,w2)+b2)

#### 第三層(code layer)

In [7]:
w3 = weight_variable([100,20],'w3')
b3 = bias_variable([20],'b3')
o3 = tf.nn.relu(tf.matmul(o2,w3)+b3)

#### 第一層(decoding)

In [8]:
w4 = weight_variable([20,100],'w4')
b4 = bias_variable([100],'b4')
o4 = tf.nn.relu(tf.matmul(o3,w4)+b4)

#### 第二層(decoding)

In [9]:
w5 = weight_variable([100,300],'w5')
b5 = bias_variable([300],'b5')
o5 = tf.nn.relu(tf.matmul(o4,w5)+b5)

#### 第三層(decoding)

In [10]:
w6 = weight_variable([300,784],'w6')
b6 = bias_variable([784],'b6')
o6 = tf.nn.relu(tf.matmul(o5,w6)+b6)

#### 損失函數與優化器

In [11]:
loss = tf.reduce_mean(tf.pow(o6-X,2))
opt = tf.train.RMSPropOptimizer(0.01).minimize(loss)

### 儲存器

In [12]:
def savemodel(sess):
    import os
    curr_pos = os.getcwd()
    if 'model' not in os.listdir():
        os.mkdirs('model')
    saver = tf.train.Saver()
    save_path = saver.save(sess, "/model/mnist_autoencoder.ckpt")
    return save_path

### 實例化執行

In [13]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(20000):
        batchx = mnist.train.next_batch(50)[0]

        if step%1000==0:
            print("Step:{},Loss:{}".format(step,loss.eval(feed_dict = {X:batchx})))
        
        sess.run(opt,feed_dict = {X:batchx})
    
    print("Test Accuracy:{}".format(sess.run(loss,feed_dict = {X:mnist.test.images})))
    
    # 製圖成果
    import numpy as np
    import matplotlib.pyplot as plt
    
    pred_out = o6.eval(feed_dict = {X:mnist.train.images})
    
    for i in range(5):
        curr_img = np.reshape(mnist.train.images[i,:],(28,28))     
        pred_img = np.reshape(pred_out[i,:],(28,28))
        
        plt.matshow(curr_img, cmap = plt.get_cmap('gray'))
        plt.matshow(pred_img, cmap = plt.get_cmap('gray'))
        
        plt.show()

Step:0,Loss:0.13384133577346802
Step:1000,Loss:0.04410703480243683
Step:2000,Loss:0.03541843220591545
Step:3000,Loss:0.03408842906355858
Step:4000,Loss:0.034828633069992065
Step:5000,Loss:0.03717229887843132
Step:6000,Loss:0.0345548540353775
Step:7000,Loss:0.03262336552143097
Step:8000,Loss:0.030949663370847702
Step:9000,Loss:0.03306087851524353
Step:10000,Loss:0.030189258977770805
Step:11000,Loss:0.029912607744336128
Step:12000,Loss:0.03019678220152855
Step:13000,Loss:0.03414012864232063
Step:14000,Loss:0.030161332339048386
Step:15000,Loss:0.036159057170152664
Step:16000,Loss:0.03198835998773575
Step:17000,Loss:0.03227417916059494
Step:18000,Loss:0.03171376883983612
Step:19000,Loss:0.03455767780542374
Test Accuracy:0.031183602288365364
